In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_propagation(self, input):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

In [3]:
class Dense(Layer):
    def __init__(self, input_dim, output_dim):
        self.weights = (np.random.rand(input_dim, output_dim) - 0.5) * 0.01
        self.bias = np.random.rand(1, output_dim) - 0.5

    def forward_propagation(self, x):
        self.input = x
        self.output = np.dot(x, self.weights) + self.bias
        return self.output

    def backward_propagation(self, next_layer_grads, lr):
        grads = np.dot(next_layer_grads, self.weights.T)
        dW = np.dot(self.input.T, next_layer_grads)
        db = np.sum(next_layer_grads, axis=0)
        
        self.weights -= lr * dW
        self.bias -= lr * db
        return grads

In [4]:
class Activation(Layer):
    def __init__(self, activation):
        self.activation = activation
        if (activation == 'tanh'):
            self.__forward_prop_fn__ = self.__tanh__
            self.__backward_prop_fn__ = self.__dtanh__
        elif (activation == 'sigmoid'):
            self.__forward_prop_fn__ = self.__sigmoid__
            self.__backward_prop_fn__ = self.__dsigmoid__
        elif (activation == 'relu'):
            self.__forward_prop_fn__ = self.__relu__
            self.__backward_prop_fn__ = self.__drelu__

    def forward_propagation(self, x):
        self.input = x
        self.output = self.__forward_prop_fn__(x)
        return self.output

    def backward_propagation(self, next_layer_grads, learning_rate):
        return self.__backward_prop_fn__(self.input) * next_layer_grads
    
    def __tanh__(self, z):
        return np.tanh(z)

    def __dtanh__(self, z):
        return 1-np.tanh(z)**2

    def __sigmoid__(self, z):
        return (1 / (1 + np.exp(-z) + 10e-8))

    def __dsigmoid__(self, z):
        s = self.__sigmoid__(z)
        return s * (1 - s)
    
    def __relu__(self, z):
        return np.maximum(z, 0)
    
    def __drelu__(self, z):
        dz = np.ones(z.shape)
        dz[self.input <= 0] = 0
        return dz

In [5]:
class Loss:
    def __init__(self, loss):
        self.loss = loss
        if (loss == 'binary_crossentropy'):
            self.forward_prop_fn = self.__binary_crossentropy__
            self.backward_prop_fn = self.__dbinary_crossentropy__
        elif (loss == 'mse'):
            self.forward_prop_fn = self.__mse__
            self.backward_prop_fn = self.__dmse__
            
    def __mse__(self, y_true, y_pred):
        return np.mean(np.power(y_true-y_pred, 2))

    def __dmse__(self, y_true, y_pred):
        return 2*(y_pred-y_true)/y_true.size
    
    def __binary_crossentropy__(self, y_true, y_pred):
        return -(1 / len(y_true)) * ((y_true * np.log(y_pred)) + ((1 - y_true) * np.log(1 - y_pred)))

    def __dbinary_crossentropy__(self, y_true, y_pred):
        return -(np.divide(y_true, y_pred) - np.divide(1 - y_true, 1 - y_pred))

In [6]:
class Model:
    def __init__(self, loss):
        self.layers = []
        self.loss = Loss(loss)

    def add(self, layer):
        self.layers.append(layer)

    def predict(self, input_data, batch_size=32):
        m = len(input_data)
        results = []
        for batch in range(0, m, batch_size):
            preds = input_data[batch:batch+batch_size]
            for layer in self.layers:
                preds = layer.forward_propagation(preds)
            results.append(preds)
        return np.array(results)

    def fit(self, X, y, epochs, lr, batch_size=32):
        m = len(X)

        for i in tqdm(range(epochs)):
            err = 0
            for batch in range(0, m, batch_size):
                preds = X[batch:batch+batch_size]
                for layer in self.layers:
                    preds = layer.forward_propagation(preds)

                err += self.loss.forward_prop_fn(y[batch:batch+batch_size], preds)

                error = self.loss.backward_prop_fn(y[batch:batch+batch_size], preds)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, lr)

            err /= m
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [7]:
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28*28)
x_train = x_train.astype('float32')
x_train /= 255
y_train = np_utils.to_categorical(y_train)

x_test = x_test.reshape(x_test.shape[0], 28*28)
x_test = x_test.astype('float32')
x_test /= 255
y_test = np_utils.to_categorical(y_test)

# Network
net = Model('mse')
net.add(Dense(28*28, 100))
net.add(Activation('tanh'))
net.add(Dense(100, 50))
net.add(Activation('tanh'))
net.add(Dense(50, 10))
net.add(Activation('tanh'))


net.fit(x_train, y_train, epochs=30, lr=0.1)

out = net.predict(x_test[0:3])
print("\npredicted values : ")
print(out)
print("true values : ")
print(y_test[0:3])

Using TensorFlow backend.


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1/30   error=0.002807
epoch 2/30   error=0.002611
epoch 3/30   error=0.002350
epoch 4/30   error=0.002045
epoch 5/30   error=0.001641
epoch 6/30   error=0.001364
epoch 7/30   error=0.001177
epoch 8/30   error=0.000951
epoch 9/30   error=0.000796
epoch 10/30   error=0.000706
epoch 11/30   error=0.000642
epoch 12/30   error=0.000593
epoch 13/30   error=0.000556
epoch 14/30   error=0.000526
epoch 15/30   error=0.000500
epoch 16/30   error=0.000478
epoch 17/30   error=0.000458
epoch 18/30   error=0.000439
epoch 19/30   error=0.000423
epoch 20/30   error=0.000407
epoch 21/30   error=0.000393
epoch 22/30   error=0.000380
epoch 23/30   error=0.000368
epoch 24/30   error=0.000357
epoch 25/30   error=0.000347
epoch 26/30   error=0.000338
epoch 27/30   error=0.000329
epoch 28/30   error=0.000321
epoch 29/30   error=0.000314
epoch 30/30   error=0.000307

predicted values : 
[[[ 3.55788555e-03  3.84869627e-03 -4.63767997e-02 -2.77789265e-02
   -1.59241049e-02  7.91942588e-03  1.42106149e-02 